In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
from torch.nn import functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#dataset
trtransform = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
tetransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trtransform)
test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tetransform)

num = len(train)
indices = list(range(num))
split = int(np.floor(0.1 * num))
train_idx, valid_idx = indices[split:], indices[:split]
np.random.seed(10)
np.random.shuffle(indices)
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, sampler=train_sampler)
validloader = torch.utils.data.DataLoader(train, batch_size=128, sampler=valid_sampler)
testloader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class SEBlock(nn.Module):
  def __init__(self, planes):
    super(SEBlock, self).__init__()
    self.avg_pool = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Sequential(nn.Linear(planes, planes // 16), nn.ReLU(inplace=True), nn.Linear(planes // 16, planes), nn.Sigmoid())

  def forward(self, x):
    b, c, _, _ = x.size()
    y = self.avg_pool(x).view(b, c)
    y = self.fc(y).view(b, c, 1, 1)
    return x * y.expand_as(x)
      
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.senet = SEBlock(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.senet(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out
      
class resnet18(nn.Module):
    def __init__(self):
        super(resnet18, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(BasicBlock, 16, 3, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 32, 3, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 64, 3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(64, 10)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

model = resnet18().to(device)

#hyperparameter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

#Train
for epoch in range(1,51):
  model.train()
  train_loss = 0.0
  val_loss=0.0
  traincorrect=0
  valcorrect = 0
  for image, label in trainloader:
    image, label = image.to(device), label.to(device)
    
    optimizer.zero_grad()
    output = model(image)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    train_loss += loss.item() / 45000
    _, pred = torch.max(output.data, 1)
    traincorrect += (pred == label).sum().item()
  model.eval()
  with torch.no_grad():
    for image, label in validloader:
      image, label = image.to(device), label.to(device)
      output = model(image)
      loss = criterion(output, label)
      val_loss += loss.item() / 5000
      _, pred = torch.max(output.data, 1)
      valcorrect += (pred == label).sum().item()
  scheduler.step()
  if epoch % 5 == 0 :
    print('Epoch:{}/{}, train_loss:{}, train_acc:{}, val_loss:{}, validation Accuracy: {}'.format(epoch, 50, train_loss, traincorrect/45000, val_loss, 100 * valcorrect / 5000))
  

#Test acc
correct = 0
tot = 0
with torch.no_grad():
  for image, label in testloader:
    image, label = image.to(device), label.to(device)
    output = model(image)
    _, pred = torch.max(output.data, 1)
    correct += (pred == label).sum().item()

print('Test Accuracy: {}'.format(100 * correct / len(testloader.dataset)))

Files already downloaded and verified
Files already downloaded and verified
